gseapy analysis of H1 triflic  
gsea algorithm: http://software.broadinstitute.org/gsea/doc/GSEAUserGuideFrame.html  
gseapy: https://media.readthedocs.org/pdf/gseapy/latest/gseapy.pdf  

In [1]:
import sys
import os
from itertools import chain
from collections import defaultdict
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('precision', 3)
pd.set_option('display.max_colwidth', -1)
import gseapy as gp
import goatools

obodag = goatools.obo_parser.GODag('go-basic.obo')

load obo file go-basic.obo
go-basic.obo: fmt(1.2) rel(2018-01-12) 49,288 GO Terms


In [2]:
sys.path.insert(0, "/home/gstupp/projects/metaproteomics")
from metaproteomics import utils
#from metaproteomics.analysis import build_loci

BASE = 'out/'
grouped_loci = utils.load(os.path.join(BASE,"grouped_loci_filt_annot.pkl.gz"))

In [3]:
def make_go2Gene_map(grouped_loci, ontology='MF'):    

    ontology_map = {'MF': 'molecular_function', 'BP': 'biological_process', 'CC': 'cellular_component'}
    
    out = defaultdict(set)    
    for l in grouped_loci:
        if 'go' in l.annotations:
            for go in l.annotations['go']:
                if obodag[go].namespace == ontology_map[ontology]:
                    out[go].add(l.cluster_id)
                    for parent in obodag[go].get_all_parents():
                        if obodag[parent].namespace == ontology_map[ontology]:
                            out[parent].add(l.cluster_id)
                
    return dict(out)

def filter_go2gene_map(go_locus):
    
    # Remove "very broad" gene sets. Arbitrary definition: gene sets that emcompass >50% of all IDs
    all_ids = set(chain(*go_locus.values()))
    go_locus = {key: value for (key, value) in go_locus.items() if len(value) / len(all_ids) <= 0.5}

    # Remove terms with less than 5 members: changed from 10 to 5 becasue small #s of proteins compared
    # to what you would find wiht genes
    go_locus = {key: value for (key, value) in go_locus.items() if len(value) >= 5}

    # Remove child terms with identical gene sets as their parents
    to_remove = set()
    for parent in go_locus.keys():
        # If child term has exact same members as parent, remove
        child_ids = [x.id for x in obodag[parent].children if x.id in go_locus.keys()]
        for child in child_ids:
            if go_locus[child] == go_locus[parent]:
                to_remove.add(child)
    go_locus = {key: value for (key, value) in go_locus.items() if key not in to_remove}

    # Remove sibling terms with identical gene sets
    to_remove = set()
    to_keep = set()
    for brother in go_locus.keys():
        to_keep.add(brother) # make sure filtered out siblings don't filter out ones we want to keep
        for parent in obodag[brother].parents:
            siblings = set([y.id for y in parent.children])
            if brother in siblings:
                siblings.remove(brother)
            for sibling in siblings:
                if sibling in go_locus.keys() and go_locus[brother] == go_locus[sibling] and not sibling in to_keep:
                    to_remove.add(sibling)
    go_locus = {key: value for (key, value) in go_locus.items() if key not in to_remove}

    return go_locus

def gomap_to_csv(go2gene, out_file = 'test.tsv'):

    out = ""
    for term, loci in go2gene.items():
        out += "{}\t".format(term)
        out += "{}\t".format(obodag[term].name)
        out += '\t'.join(list(map(str,loci)))
        out += '\n'
        
    with open(out_file, 'w') as fout:
        fout.write(out)

In [4]:
def run_go_gsea(rank_df, g2g_map, seed, outdir='tmp'):
    """
    A ranked df and go2gene mapping returns the result dataframe for GSEA against all go-Terms
    
    loci must be grouped such that avg_ratio and p-values are correct for 1 phenotype
    see rt_unenr_grouped_loci above for example
    """
    import gseapy as gp
        
    # save the go 2 gene map, since gseapy doesn't seem to be able to use one already in memory
    gomap_to_csv(g2g_map, 'temp.gmt')
    
    res = gp.prerank(rnk=rank_df, gene_sets='temp.gmt', outdir=outdir, min_size = 5, max_size=500, 
                     permutation_n = 10000, graph_num = 20, seed=seed)
      
    def get_go_name(term):
        return obodag[term].name
    
    res['name'] = res.index.map(get_go_name)
    
    return res.sort_values('nes', ascending=False)

def plot_gsea_result(row, rank):
    return gp.plot.gsea_plot(rank, row['name'], row.hit_index, row.nes, row.pval, row.fdr, row.rank_ES, phenoPos='Tcell', phenoNeg='RAG')

In [5]:
mf_map = make_go2Gene_map(grouped_loci)
mf_map_f = filter_go2gene_map(mf_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(mf_map), len(mf_map_f)))

bp_map = make_go2Gene_map(grouped_loci, 'BP')
bp_map_f = filter_go2gene_map(bp_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(bp_map), len(bp_map_f)))

cc_map = make_go2Gene_map(grouped_loci, 'CC')
cc_map_f = filter_go2gene_map(cc_map)

print('Unfiltered: {}\tFiltered: {}'.format(len(cc_map), len(cc_map_f)))

Unfiltered: 474	Filtered: 210
Unfiltered: 602	Filtered: 232
Unfiltered: 95	Filtered: 48


In [6]:
out_dir = "sonic_sup_whole_gsea"
df = pd.read_csv(os.path.join(BASE,"sonic_sup_v_whole_deseq_results.csv"))
df = df[(df.padj.abs()<=0.2)]
df['log2FoldChange'] = -1 * df['log2FoldChange']
rank_df = df[['Unnamed: 0', 'log2FoldChange']].rename(columns={'Unnamed: 0': 'gene_name', 'log2FoldChange': 'rank'})
rank_df = rank_df.sort_values('rank').reset_index(drop=True)

In [7]:
rank_df.head()

,gene_name,rank
0,62761516,-9.150
1,36974013,-8.926
2,39268506,-8.762
3,62236100,-8.731
4,63322719,-8.643


In [8]:
mf_res = run_go_gsea(rank_df, mf_map_f, seed=1111, outdir=out_dir)
bp_res = run_go_gsea(rank_df, bp_map_f, seed=1111, outdir=out_dir)
mf_rt = mf_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
mf_control = mf_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)
bp_rt = bp_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
bp_control = bp_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2018-02-13 11:51:09,963 Parsing data files for GSEA.............................
/home/gstupp/.local/lib/python3.5/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2018-02-13 11:51:11,306 0088 gene_sets used for further statistical testing.....
2018-02-13 11:51:11,307 Start to run GSEA...Might take a while..................
2018-02-13 11:58:40,731 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2018-02-13 11:58:48,504 Congratulations...GSEAPY run successfully...............
2018-02-13 11:58:48,525 Parsing data files for GSEA.............................
2018-02-13 11:58:49,951 0094 gene_sets used for further 

In [9]:
mf_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0004611,0.688,4.788,0.000e+00,0.000e+00,81,79,"[41688438, 65362612, 40582524, 65031514, 53629697, 45517643, 49026941, 58388428, 42660247, 65423787, 62078189, 43556646, 58527538, 20942064, 64982050, 65743137, 70403312, 39426259, 67866969, 41914852, 40656833, 43734696, 45084143, 21022171, 57769961, 66824230, 62091010, 58317121, 54789106, 42606314, 66199306, 67249520, 39707926, 43394027, 66958425, 10848066, 51589159, 13308883, 62033622, 166396878, 166285214, 52151281, 58132764, 64743418, 51101110, 48156244, 13675824, 64070550, 22764913, 17563766, 44526011, 62318683, 20922359, 20910477, 58886235, 50127458, 43129949, 40914457, 48632614, 4706576, 57681464, 25957523, 45349478, 20910442, 64920279, 49784426, 56332816, 48413248, 52919310, 22182132, 45625937, 56366728, 51175066, 28556640, 37145660, 167777308, 56925762, 62322784, 45819353]",phosphoenolpyruvate carboxykinase activity
GO:0004612,0.691,4.483,0.000e+00,0.000e+00,65,63,"[65362612, 40582524, 65031514, 45517643, 58388428, 65423787, 43556646, 58527538, 64982050, 65743137, 67866969, 43734696, 21022171, 57769961, 66824230, 58317121, 54789106, 42606314, 66199306, 67249520, 10848066, 51589159, 13308883, 62033622, 166396878, 166285214, 52151281, 58132764, 64743418, 51101110, 48156244, 13675824, 64070550, 22764913, 17563766, 44526011, 62318683, 20922359, 20910477, 58886235, 50127458, 43129949, 40914457, 48632614, 4706576, 57681464, 25957523, 45349478, 20910442, 64920279, 49784426, 56332816, 48413248, 52919310, 22182132, 45625937, 56366728, 51175066, 28556640, 37145660, 167777308, 56925762, 62322784]",phosphoenolpyruvate carboxykinase (ATP) activity
GO:0016831,0.607,4.199,0.000e+00,0.000e+00,86,83,"[41688438, 65362612, 40582524, 65031514, 53629697, 45517643, 49026941, 58388428, 42660247, 65423787, 62078189, 43556646, 58527538, 20942064, 64982050, 65743137, 70403312, 39426259, 67866969, 41914852, 40656833, 43734696, 45084143, 21022171, 57769961, 66824230, 62091010, 58317121, 54789106, 42606314, 66199306, 67249520, 39707926, 43394027, 66958425, 10848066, 51589159, 13308883, 62033622, 166396878, 166285214, 52151281, 58132764, 64743418, 51101110, 48156244, 13675824, 64070550, 22764913, 17563766, 44526011, 62318683, 20922359, 20910477, 58886235, 50127458, 43129949, 40914457, 48632614, 4706576, 57681464, 25957523, 45349478, 20910442, 64920279, 49784426, 56332816, 48413248, 52919310, 22182132, 45625937, 56366728, 51175066, 28556640, 37145660, 167777308, 651398, 56925762, 62322784, 45819353, 67390440, 16675474, 14278226]",carboxy-lyase activity
GO:0016852,0.701,4.170,0.000e+00,0.000e+00,46,45,"[43325697, 54643075, 54690680, 58799028, 37318483, 52019133, 41242315, 62022650, 60198961, 62651633, 62344516, 39172859, 167609360, 62772745, 51661852, 65472615, 62887159, 41318454, 65884410, 62271819, 64380879, 45799801, 62037065, 62489518, 61238641, 66785143, 60430276, 50202327, 166044631, 62374554, 59069679, 65468006, 51766415, 11762451, 51299489, 36241782, 64532907, 62125341, 66312194, 167709467, 62225985, 62450424, 65349525, 50069915, 63687761]",sirohydrochlorin cobaltochelatase activity
GO:0016814,0.734,3.680,0.000e+00,0.000e+00,48,26,"[64861784, 166416342, 65420785, 59296033, 44484649, 47630323, 66430161, 21312369, 64976161, 166986795, 55182519, 51462612, 60663896, 64363232, 64362864, 59919098, 66809311, 64905338, 64166940, 64381315, 8047714, 44223410, 56263791, 66491142, 37615398, 56610477]","hydrolase activity, acting on carbon-nitrogen (but not peptide) bonds, in cyclic amidines"
GO:0004807,0.488,3.166,0.000e+00,0.000e+00,74,61,"[64195026, 64703224, 65264616, 64841796, 64133206, 64498996, 65106338, 63492044, 29389525, 64885716, 65018571, 65032287, 64455320, 57980588, 65046198, 65078746, 54457320, 65081691, 65087679, 65097705, 67240771, 54306173, 64473889, 64875018, 36734803, 36393272, 22209850, 17395134, 13754598, 7983883, 51821008, 166334753, 43017203, 42387603, 65065317, 7684206, 65235043, 65089145, 168199724, 8956920, 5720

In [10]:
mf_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0016880,-0.719,-4.600,0.000e+00,0.000e+00,354,346,"[61924909, 61840025, 8032062, 8047113, 8195829, 7973249, 21467527, 9048587, 8402422, 7962146, 9270396, 9723318, 10286900, 10365777, 10446427, 21419353, 21397527, 10510823, 8401762, 21570364, 10899019, 21974580, 6167382, 7091833, 7267974, 7286941, 7366547, 22217450, 7369045, 7430033, 7435790, 7491772, 7530963, 22163933, 7583511, 22074862, 7607889, 21392141, 7678047, 7931537, 21974789, 21253138, 18124263, 18047662, 10687793, 17953809, 10689796, 16841849, 15819993, 15167402, 14433022, 13726185, 10715532, 10730846, 13602922, 10773288, 12031731, 18594862, 18885106, 64783339, 64602626, 21041123, 11533029, 20968127, 20872157, 20871392, 10531299, 20636481, 21369340, 20633999, 20333901, 20329663, 10533506, 6122871, 20325100, 20014313, 18950246, 64746940, 20604552, 61896244, 5798443, 166788817, 166598674, 166455421, 166316037, 166187048, 166096580, 165825325, 165806817, 166796953, 166917403, 61897200, 168122802, 167807103, 167742008, 167737667, 167730639, 167686814, 167650683, 167565226, ...]",acid-ammonia (or amide) ligase activity
GO:0016879,-0.682,-4.393,0.000e+00,0.000e+00,431,382,"[38867144, 62250933, 20968243, 64856944, 64164012, 47981596, 65135677, 62018486, 21347547, 61964112, 66025336, 62113144, 62571187, 67213458, 38210069, 36238237, 68597221, 65428360, 64709852, 51305083, 41896895, 17571629, 62223265, 62561967, 67118703, 52291787, 62517803, 21823200, 64467565, 45677108, 60433306, 56040554, 51213248, 65510618, 62574969, 61924909, 61840025, 8032062, 8047113, 8195829, 7973249, 21467527, 9048587, 8402422, 7962146, 9270396, 9723318, 10286900, 10365777, 10446427, 21419353, 21397527, 10510823, 8401762, 21570364, 10899019, 21974580, 6167382, 7091833, 7267974, 7286941, 7366547, 22217450, 7369045, 7430033, 7435790, 7491772, 7530963, 22163933, 7583511, 22074862, 7607889, 21392141, 7678047, 7931537, 21974789, 21253138, 18124263, 18047662, 10687793, 17953809, 10689796, 16841849, 15819993, 15167402, 14433022, 13726185, 10715532, 10730846, 13602922, 10773288, 12031731, 18594862, 18885106, 64783339, 64602626, 21041123, 11533029, 20968127, 20872157, ...]","ligase activity, forming carbon-nitrogen bonds"
GO:0097747,-0.509,-3.219,0.000e+00,0.000e+00,318,298,"[28233603, 61586849, 45038305, 54892854, 66169230, 62562027, 48307887, 62070529, 56757236, 50692088, 62018988, 56945065, 4099172, 18109239, 53190058, 53072938, 10815512, 22238189, 39275703, 9012457, 21126076, 9215855, 7627337, 41751957, 63996097, 64034734, 63856881, 63974364, 6384290, 18193491, 63995453, 63857026, 63800039, 62092597, 63849268, 13596265, 18214391, 17625098, 7289884, 61719675, 61601524, 41165392, 51691431, 62155980, 61604840, 167412647, 61631863, 61586861, 21126049, 22236358, 5535467, 4081737, 28962119, 3704668, 31058828, 8692550, 8887938, 10252913, 11928233, 13664144, 31893283, 37387955, 62823286, 62874585, 63410001, 76298777, 165818124, 165825861, 5887279, 61079203, 8029150, 61595851, 61593743, 61736143, 61598320, 61589098, 36374512, 28430821, 21254405, 10774345, 6023174, 61782419, 168043212, 61607113, 21880278, 59642867, 58926833, 52385306, 15890002, 17201391, 20871177, 7440434, 4172233, 165694804, 20901943, 167428759, 16830531, 2011772, 166188741, 165965080, ...]",RNA polymerase activity
GO:0003677,-0.492,-3.186,0.000e+00,0.000e+00,432,399,"[2041002, 1822783, 2058497, 2057470, 1958375, 209005, 68051075, 1818671, 2456528, 1158404, 1023145, 7173479, 7697530, 30751374, 63933277, 21230260, 28233603, 61586849, 45038305, 66169230, 62562027, 48307887, 62070529, 56757236, 30842656, 52689644, 63340726, 62869708, 50692088, 62018988, 56945065, 47583840, 4099172, 18109239, 6205815, 7009404, 68704849, 167587023, 68582616, 167775873, 167395572, 166037070, 68206104, 68150755, 67950579, 68671309, 67819674, 67744907, 67353000, 54869360, 68283504, 68757205, 68398056, 69263052, 166395738, 69322165, 69216024, 69422003, 69982823, 165788228, 10815512, 22238189, 39275703, 90

In [11]:
bp_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0040011,0.710,4.741,0.000,0.000e+00,86,69,"[62398453, 62543065, 62698072, 62881914, 62350005, 62450054, 64272473, 62769590, 62343668, 62271880, 62182764, 22255856, 62285751, 63245372, 62964422, 57741843, 62342496, 65192787, 67423079, 166851534, 13433916, 22625707, 42831732, 62417891, 55916126, 56924487, 62085029, 62923262, 28438217, 64855697, 63865661, 62405618, 63669394, 63452851, 62926584, 62853162, 62849941, 62510437, 21809431, 62265597, 38214195, 62582614, 43730808, 62157933, 45950065, 63205708, 63306571, 49361447, 1886631, 39745655, 23658537, 62766323, 62838017, 23658535, 63093125, 64376861, 63144665, 63039819, 48844574, 62966887, 49492210, 63798592, 65147516, 63489079, 65866265, 167570053, 64167744, 65968008, 68570136]",locomotion
GO:0006928,0.710,4.734,0.000,0.000e+00,86,69,"[62398453, 62543065, 62698072, 62881914, 62350005, 62450054, 64272473, 62769590, 62343668, 62271880, 62182764, 22255856, 62285751, 63245372, 62964422, 57741843, 62342496, 65192787, 67423079, 166851534, 13433916, 22625707, 42831732, 62417891, 55916126, 56924487, 62085029, 62923262, 28438217, 64855697, 63865661, 62405618, 63669394, 63452851, 62926584, 62853162, 62849941, 62510437, 21809431, 62265597, 38214195, 62582614, 43730808, 62157933, 45950065, 63205708, 63306571, 49361447, 1886631, 39745655, 23658537, 62766323, 62838017, 23658535, 63093125, 64376861, 63144665, 63039819, 48844574, 62966887, 49492210, 63798592, 65147516, 63489079, 65866265, 167570053, 64167744, 65968008, 68570136]",movement of cell or subcellular component
GO:0006766,0.701,4.234,0.000,0.000e+00,53,47,"[43325697, 54643075, 54690680, 58799028, 37318483, 52019133, 41242315, 62022650, 60198961, 62651633, 62344516, 39172859, 167609360, 62772745, 51661852, 65472615, 62887159, 41318454, 65884410, 62271819, 64380879, 45799801, 62037065, 62489518, 61238641, 66785143, 60430276, 50202327, 166044631, 62374554, 66504785, 59069679, 65468006, 51766415, 11762451, 51299489, 36241782, 64532907, 2412903, 62125341, 66312194, 167709467, 62225985, 62450424, 65349525, 50069915, 63687761]",vitamin metabolic process
GO:0033013,0.701,4.199,0.000,0.000e+00,46,45,"[43325697, 54643075, 54690680, 58799028, 37318483, 52019133, 41242315, 62022650, 60198961, 62651633, 62344516, 39172859, 167609360, 62772745, 51661852, 65472615, 62887159, 41318454, 65884410, 62271819, 64380879, 45799801, 62037065, 62489518, 61238641, 66785143, 60430276, 50202327, 166044631, 62374554, 59069679, 65468006, 51766415, 11762451, 51299489, 36241782, 64532907, 62125341, 66312194, 167709467, 62225985, 62450424, 65349525, 50069915, 63687761]",tetrapyrrole metabolic process
GO:0046364,0.527,3.878,0.000,0.000e+00,109,103,"[40327086, 51457463, 38362906, 41688438, 65362612, 40582524, 65031514, 53629697, 29430347, 13670649, 45517643, 49026941, 58388428, 42660247, 65423787, 62078189, 43556646, 58527538, 20942064, 64982050, 65743137, 70403312, 39426259, 67866969, 41914852, 40656833, 43734696, 45084143, 21022171, 57769961, 66824230, 62091010, 58317121, 54789106, 42606314, 66199306, 67249520, 66988806, 39707926, 43394027, 66958425, 10848066, 51589159, 13308883, 57125904, 62033622, 166396878, 166285214, 52151281, 58132764, 64743418, 51673203, 51101110, 48156244, 13675824, 64070550, 22764913, 55816901, 17563766, 44526011, 62318683, 20922359, 20910477, 58886235, 50127458, 43129949, 40914457, 48632614, 4706576, 57681464, 25957523, 45349478, 20910442, 43189595, 64920279, 49784426, 56332816, 48413248, 52919310, 22182132, 45625937, 56366728, 51175066, 28556640, 37145660, 167777308, 68018518, 60931305, 65447256, 38603820, 63986036, 56925762, 62322784, 42399038, 45917122, 60662052, 65725341, 65029251, 63597941, 39787083, ...]",monosaccharide biosynthetic process
GO:0016051,0.436,3.243,0.000,0.000e+00,119,112,"[40327086, 51457463, 38362906, 41688438, 65362612, 40582524, 65031514, 53629697, 29430347, 13670649, 45517643, 49026941, 58388428, 42660247, 65423787, 62078189, 43556646, 58527538, 2

In [13]:
bp_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0032774,-0.507,-3.197,0.000e+00,0.000e+00,320,300,"[28233603, 61586849, 45038305, 54892854, 66169230, 62562027, 48307887, 62070529, 56757236, 50692088, 62018988, 56945065, 4099172, 18109239, 53190058, 53072938, 60582086, 10815512, 22238189, 39275703, 9012457, 21126076, 9215855, 7627337, 41751957, 63996097, 64034734, 63856881, 63974364, 6384290, 18193491, 63995453, 63857026, 63800039, 62092597, 63849268, 13596265, 18214391, 17625098, 7289884, 61719675, 61601524, 41165392, 51691431, 62155980, 61604840, 167412647, 61631863, 61586861, 21126049, 22236358, 5535467, 4081737, 28962119, 3704668, 31058828, 8692550, 8887938, 10252913, 11928233, 13664144, 31893283, 37387955, 62823286, 62874585, 63410001, 76298777, 165818124, 165825861, 5887279, 61079203, 8029150, 61595851, 61593743, 61736143, 61598320, 61589098, 36374512, 28430821, 21254405, 10774345, 6023174, 61782419, 168043212, 61607113, 21880278, 59642867, 58926833, 52385306, 15890002, 17201391, 20871177, 7440434, 4172233, 165694804, 20901943, 167428759, 16830531, 2011772, 166188741, ...]",RNA biosynthetic process
GO:0090304,-0.482,-3.123,0.000e+00,0.000e+00,451,424,"[20322451, 7173479, 7697530, 30751374, 63933277, 21230260, 28233603, 61586849, 18885724, 167282458, 65389288, 64506172, 167277868, 62726266, 62393365, 52576745, 52481957, 28962162, 11791224, 45038305, 54892854, 45031759, 66169230, 62562027, 48307887, 62070529, 56757236, 30842656, 52689644, 63340726, 62869708, 66440034, 66068248, 54345745, 32783706, 63271051, 41750634, 44453573, 48472643, 43335971, 58267232, 62491289, 62542342, 62432381, 50692088, 6789864, 62018988, 56945065, 47583840, 56002051, 4099172, 18109239, 6205815, 7009404, 21898869, 68704849, 167587023, 68582616, 167775873, 167395572, 166037070, 68206104, 68150755, 67950579, 68671309, 67819674, 67744907, 67353000, 54869360, 68283504, 68757205, 68398056, 69263052, 166395738, 69322165, 69216024, 69422003, 69982823, 165788228, 53190058, 53072938, 60582086, 10815512, 22238189, 39275703, 9012457, 21126076, 9215855, 7627337, 41751957, 62472987, 8291745, 63996097, 64034734, 28669037, 63856881, 63974364, 6384290, 18193491, 62150994, ...]",nucleic acid metabolic process
GO:0016070,-0.483,-3.107,0.000e+00,0.000e+00,390,366,"[28233603, 61586849, 18885724, 167282458, 65389288, 64506172, 167277868, 62726266, 62393365, 52576745, 52481957, 28962162, 11791224, 45038305, 54892854, 45031759, 66169230, 62562027, 48307887, 62070529, 56757236, 66440034, 66068248, 54345745, 32783706, 63271051, 41750634, 44453573, 48472643, 43335971, 58267232, 62491289, 62542342, 62432381, 50692088, 6789864, 62018988, 56945065, 56002051, 4099172, 18109239, 21898869, 53190058, 53072938, 60582086, 10815512, 22238189, 39275703, 9012457, 21126076, 9215855, 7627337, 41751957, 62472987, 8291745, 63996097, 64034734, 28669037, 63856881, 63974364, 6384290, 18193491, 62150994, 62280444, 62282436, 62432270, 166993709, 63995453, 49798158, 63857026, 63800039, 62092597, 63849268, 17592682, 13596265, 18214391, 17625098, 7289884, 61719675, 61601524, 41165392, 51691431, 62155980, 61604840, 167412647, 61631863, 61586861, 21126049, 22236358, 5535467, 4081737, 28962119, 3704668, 31058828, 8692550, 8887938, 10252913, 11928233, 13664144, 31893283, ...]",RNA metabolic process
GO:0034654,-0.443,-2.866,0.000e+00,0.000e+00,483,402,"[64861784, 166416342, 65420785, 62229406, 38262356, 58745702, 59296033, 44484649, 48850436, 62211247, 42697888, 64814712, 47630323, 66430161, 21312369, 64976161, 166986795, 55182519, 51462612, 60663896, 64363232, 64362864, 59919098, 66809311, 64905338, 64166940, 64381315, 8047714, 44223410, 56263791, 66491142, 37615398, 56610477, 44035050, 66911439, 51908457, 167911272, 50219646, 37081817, 62475357, 46925916, 62665934, 7441750, 28233603, 61586849, 1813814, 47320492, 45038305, 54892854, 66169230, 62562027, 48307887, 62070529, 56757236, 5200631, 50692088, 62018988, 56945065, 4099172, 62517803, 21823200, 32363526, 18109239, 44962491, 

In [14]:
cc_res = run_go_gsea(rank_df, cc_map_f, seed=1111, outdir=out_dir)
cc_rt = cc_res.query('nes > 0 and pval < 0.05').sort_values('nes', ascending=False)
cc_control = cc_res.query('nes < 0 and pval < 0.05').sort_values('nes', ascending=True)

2018-02-13 16:55:00,062 Parsing data files for GSEA.............................
/home/gstupp/.local/lib/python3.5/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2018-02-13 16:55:00,367 0020 gene_sets used for further statistical testing.....
2018-02-13 16:55:00,368 Start to run GSEA...Might take a while..................
2018-02-13 16:56:42,374 Start to generate gseapy reports, and produce figures...
/usr/lib/python3/dist-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "
2018-02-13 16:56:49,540 Congratulations...GSEAPY run successfully...............


In [15]:
cc_rt

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0042995,0.710,4.731,0.0,0.000e+00,85,69,"[62398453, 62543065, 62698072, 62881914, 62350005, 62450054, 64272473, 62769590, 62343668, 62271880, 62182764, 22255856, 62285751, 63245372, 62964422, 57741843, 62342496, 65192787, 67423079, 166851534, 13433916, 22625707, 42831732, 62417891, 55916126, 56924487, 62085029, 62923262, 28438217, 64855697, 63865661, 62405618, 63669394, 63452851, 62926584, 62853162, 62849941, 62510437, 21809431, 62265597, 38214195, 62582614, 43730808, 62157933, 45950065, 63205708, 63306571, 49361447, 1886631, 39745655, 23658537, 62766323, 62838017, 23658535, 63093125, 64376861, 63144665, 63039819, 48844574, 62966887, 49492210, 63798592, 65147516, 63489079, 65866265, 167570053, 64167744, 65968008, 68570136]",cell projection
GO:0044463,0.726,4.672,0.0,0.000e+00,73,60,"[62398453, 62543065, 62698072, 62881914, 62350005, 62450054, 64272473, 62769590, 62343668, 62271880, 62182764, 22255856, 62285751, 63245372, 62964422, 166851534, 13433916, 22625707, 42831732, 62417891, 55916126, 56924487, 62085029, 62923262, 28438217, 64855697, 63865661, 62405618, 63669394, 63452851, 62926584, 62853162, 62849941, 62510437, 21809431, 62265597, 38214195, 62582614, 62157933, 63205708, 63306571, 49361447, 1886631, 39745655, 23658537, 62766323, 62838017, 23658535, 63093125, 64376861, 63144665, 63039819, 48844574, 62966887, 49492210, 63798592, 65147516, 63489079, 65866265, 167570053]",cell projection part
GO:0005886,0.722,4.340,0.0,0.000e+00,66,46,"[13445022, 38185668, 63133556, 64360880, 62974897, 33842604, 48193770, 59182599, 47501503, 63101601, 62377777, 60186186, 47630348, 63001215, 41499392, 56236083, 46173499, 48858203, 63042548, 66579509, 45717185, 63078668, 31260831, 63112593, 66571595, 53716631, 46410130, 166005029, 63495297, 55212582, 64128095, 63055964, 63188988, 67439647, 33471949, 58120150, 64649372, 62788046, 39552245, 44142111, 39280970, 37483090, 62049799, 63059959, 66484337, 37439011]",plasma membrane
GO:1904949,0.529,3.004,0.0,0.000e+00,58,39,"[56076521, 62946388, 63332162, 13203808, 12135439, 6924770, 22562105, 64995594, 6569127, 10829039, 56215621, 66732966, 40852561, 36311677, 58930696, 10689069, 5711881, 77351967, 28360306, 17587826, 15700990, 63363572, 63452797, 47513944, 28240379, 63205501, 40726507, 40674602, 27931540, 40686252, 66502215, 66602623, 8187004, 28273397, 4596523, 54544767, 41124596, 8861059, 5062281]",ATPase complex
GO:0044459,0.529,2.994,0.0,0.000e+00,58,39,"[56076521, 62946388, 63332162, 13203808, 12135439, 6924770, 22562105, 64995594, 6569127, 10829039, 56215621, 66732966, 40852561, 36311677, 58930696, 10689069, 5711881, 77351967, 28360306, 17587826, 15700990, 63363572, 63452797, 47513944, 28240379, 63205501, 40726507, 40674602, 27931540, 40686252, 66502215, 66602623, 8187004, 28273397, 4596523, 54544767, 41124596, 8861059, 5062281]",plasma membrane part
GO:1990351,0.529,2.989,0.0,0.000e+00,58,39,"[56076521, 62946388, 63332162, 13203808, 12135439, 6924770, 22562105, 64995594, 6569127, 10829039, 56215621, 66732966, 40852561, 36311677, 58930696, 10689069, 5711881, 77351967, 28360306, 17587826, 15700990, 63363572, 63452797, 47513944, 28240379, 63205501, 40726507, 40674602, 27931540, 40686252, 66502215, 66602623, 8187004, 28273397, 4596523, 54544767, 41124596, 8861059, 5062281]",transporter complex
GO:0016020,0.285,2.252,0.0,4.269e-05,221,137,"[61717144, 13445022, 61655642, 61935358, 38185668, 63133556, 64360880, 62974897, 33842604, 51621514, 48193770, 59182599, 63471490, 62054139, 47501503, 63101601, 62377777, 63948514, 60186186, 47630348, 38216270, 63001215, 41499392, 37021135, 53943329, 66043038, 56236083, 53989012, 46173499, 48858203, 61530003, 63042548, 66579509, 59156293, 53598491, 39754209, 42028162, 61671561, 51626281, 36220708, 45717185, 45444248, 14080080, 63078668, 42005004, 31260831, 63112593, 66571595, 53716631, 46410130, 58048009, 46514055, 166005029, 63495297, 55212582, 64128095, 63055964, 63188988, 15491565, 6743964

In [16]:
cc_control

,es,nes,pval,fdr,gene_set_size,matched_size,genes,name
Term,,,,,,,,
GO:0044391,-0.435,-2.677,0.0,0.000e+00,250,232,"[166846359, 58033748, 29219123, 66047049, 65570045, 10689110, 28857394, 67104490, 67469680, 21326718, 65717267, 64021749, 22810887, 166666623, 65477380, 29213833, 27700985, 5107176, 63807427, 67230263, 66875237, 23714033, 14081574, 5746583, 14603981, 64503554, 47926867, 31549327, 46818577, 19011820, 31255291, 54181584, 55004337, 65976369, 47615003, 36401449, 18689097, 23587815, 38785731, 67859417, 45975512, 66402327, 56106808, 14150438, 167706979, 8700996, 7522455, 29612526, 67061811, 166807349, 168017014, 67389484, 29341916, 4774708, 20720618, 74406712, 82054302, 7259645, 66350717, 66541818, 19073235, 65354409, 6240736, 28544550, 33105645, 65409994, 6085738, 64615908, 64667783, 64853626, 166456781, 67094509, 7290535, 21828346, 38125758, 65886657, 21335616, 29757347, 65290575, 65595228, 29972795, 166757246, 64522941, 168226647, 27851959, 60350763, 16039210, 65408656, 14890274, 64078191, 165795593, 168064143, 64122877, 64937458, 65956045, 64744508, 18692349, 64037691, 66308665, 66591237, ...]",ribosomal subunit
GO:0015935,-0.436,-2.601,0.0,0.000e+00,180,175,"[166846359, 58033748, 29219123, 66047049, 65570045, 10689110, 28857394, 67104490, 21326718, 65717267, 64021749, 22810887, 166666623, 65477380, 29213833, 27700985, 5107176, 67230263, 66875237, 23714033, 14081574, 5746583, 14603981, 46818577, 19011820, 31255291, 54181584, 55004337, 65976369, 47615003, 38785731, 45975512, 66402327, 56106808, 167706979, 7522455, 29612526, 67061811, 166807349, 168017014, 67389484, 29341916, 4774708, 20720618, 74406712, 82054302, 7259645, 66350717, 66541818, 6085738, 64615908, 64667783, 64853626, 166456781, 67094509, 7290535, 21828346, 65886657, 21335616, 64522941, 168226647, 27851959, 60350763, 16039210, 65408656, 14890274, 64078191, 165795593, 168064143, 64122877, 65956045, 64744508, 64037691, 66308665, 66591237, 7381866, 12255887, 14783236, 10815303, 30844263, 66920578, 65156191, 65488847, 4343484, 63998974, 64115139, 20106802, 67648516, 66050469, 15743412, 64239927, 18720605, 5107198, 66756705, 28225353, 63962734, 66324910, 66196239, 66242221, 9904239, ...]",small ribosomal subunit
GO:0005622,-0.400,-2.590,0.0,0.000e+00,474,420,"[67600784, 37911383, 23712773, 7822625, 67530100, 40801738, 22855140, 45552631, 69853796, 23028276, 23390113, 21299373, 12268875, 57729363, 63212163, 166846359, 61919677, 27258242, 61886286, 61907404, 58033748, 29219123, 21327765, 20995791, 46820147, 66047049, 67795556, 7529421, 65570045, 10689110, 39439162, 4009240, 28857394, 7569326, 32784939, 67104490, 21902328, 4100571, 7092868, 67469680, 21326718, 29940019, 65717267, 36785606, 64021749, 22810887, 61895769, 70180230, 20322451, 166666623, 22163880, 61917199, 16613806, 29768242, 65477380, 68010124, 61908356, 29213833, 61893464, 27700985, 67963058, 63035382, 23619953, 5107176, 61913890, 62860561, 63807427, 3972270, 4344665, 23587484, 66802302, 23714033, 14081574, 5746583, 14603981, 66405184, 57932979, 68127142, 68307507, 74304065, 20904247, 64503554, 20813343, 47926867, 62535487, 67981082, 31549327, 64312859, 67499229, 70329620, 67855986, 44292295, 40441269, 20275740, 8092444, 46818577, 28624369, 24075428, 10276368, 67569222, ...]",intracellular
GO:0044446,-0.404,-2.514,0.0,0.000e+00,283,261,"[61530003, 165642734, 61500275, 61468943, 165644696, 165642706, 165677957, 61506894, 61501151, 61499720, 61518541, 61501147, 61519450, 2041002, 1822783, 2058497, 2057470, 1958375, 209005, 68051075, 1818671, 166846359, 58033748, 29219123, 66047049, 65570045, 10689110, 2456528, 28857394, 67104490, 1158404, 1023145, 67469680, 21326718, 65717267, 64021749, 22810887, 166666623, 65477380, 29213833, 27700985, 5107176, 63807427, 67230263, 66875237, 23714033, 14081574, 5746583, 14603981, 64503554, 47926867, 31549327, 46818577, 61504270, 19011820, 31255291, 54181584, 55004337, 65976369, 47615003, 36401449, 18689097, 23587815, 38785731, 67859417, 45975512, 66402327, 56106808,